In [ ]:
data_names = ['mRNA','miRNA','MNIST','Fashion-MNIST','Muraro','ALM','AMB','VISP', 'Baron_Human', 'Baron_Mouse', 'Segerstolpe', \
'Tcell-medicine', 'Zhengmix8eq','Xin', 'TM']

from cplearn.corespect import Corespect
import data_call

results = {}
for d_name in data_names:
    # Load Data
    #dpath = 'D:/Data/Data/'
    X,label=data_call.get_dataset(d_name, mode='PCA')#, dpath_in = dpath)
    hmap={}
    t=0
    for i in sorted(set(label)):
        hmap[i]=t
        t+=1
    new_label= [hmap[i] for i in label]
    y= new_label

    true_k = len(set(y))
    print(d_name) 
    # Initialize Corespect with the synthetic data
    corespect = Corespect()
        
    for algos in ['k_means', 'spectral_clustering',  'hdbscan']:
        # Fit the model
        if algos in ['louvain', 'hdbscan']:
            pred_labels = corespect.fit_predict(X = X, cluster_algo= algos)
        else:
            pred_labels = corespect.fit_predict(X = X, cluster_algo= algos, k = true_k)
        
        # for name, val in vars(corespect).items():
        #     if name.startswith('_'):
        #         continue
        #     #print(f"{name}: {val}")
        #     print(f"{name}")



        #calculate NMI and ARI
        from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
        nmi = normalized_mutual_info_score(y, pred_labels)
        ari = adjusted_rand_score(y, pred_labels)
        print(f"NMI: {nmi:.4f}, ARI: {ari:.4f}")

In [ ]:
# Plot the X and y 
def plot_data(X, y):
    """
    Plot the data using matplotlib.
    """
    import matplotlib.pyplot as plt
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=10)
    plt.title('Synthetic Data')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.colorbar()
    plt.show()
plot_data(X,y)

In [ ]:

"""
Test the Corespect class with synthetic data.
"""


# Initialize Corespect with the synthetic data
corespect = Corespect()
    
for algos in ['k_means', 'spectral_clustering', 'louvain', 'hdbscan']:
    # Fit the model
    if algos in ['louvain', 'hdbscan']:
        pred_labels = corespect.fit_predict(X = X, cluster_algo= algos)
    else:
        pred_labels = corespect.fit_predict(X = X, cluster_algo= algos, k = 5)
    
    # for name, val in vars(corespect).items():
    #     if name.startswith('_'):
    #         continue
    #     #print(f"{name}: {val}")
    #     print(f"{name}")



    #calculate NMI and ARI
    from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
    nmi = normalized_mutual_info_score(y, pred_labels)
    ari = adjusted_rand_score(y, pred_labels)
    print(f"NMI: {nmi:.4f}, ARI: {ari:.4f}")


